In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from keras import backend as K
K.clear_session()          # 清除之前会话，防止jupyter缓存影响

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)   # 加载数据（若无则从网上下载，如有则加载本地）

def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = sess.run(prediction, feed_dict={xs:v_xs, keep_prob:1})
    correct_prediction = tf.equal(tf.argmax(y_pre, 1), tf.argmax(v_ys, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs:v_xs, ys:v_ys, keep_prob:1})
    return result

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)    
    return tf.Variable(initial)
 
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    '''卷积神经网络层
    x：图片值
    W：权重
    '''
    # strides[1, x_movement, y_movement, 1]
    # 水平方向跨度为1，数值方向跨度为1；VALID边界不填充0（抽取长宽比原图片小），SAME边界填充0（抽取长宽与原图片相同）
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME') 

def max_pool_2x2(x):
    '''Pooling，防止跨度太大，丢失太多信息'''
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# 定义传入值
xs = tf.placeholder(tf.float32, [None, 784])   # 28×28
ys = tf.placeholder(tf.float32, [None, 10])  
keep_prob = tf.placeholder(tf.float32)
x_image = tf.reshape(xs, [-1, 28, 28, 1])  # 图片为灰白，channal只有一个；若图片为彩色，则channal有3个

# conv1 layer
W_conv1 = weight_variable([5, 5, 1, 32])  # patch 5×5，in size 1（厚度），out size 32
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1)+b_conv1)   # output size 28×28×32
h_pool1 = max_pool_2x2(h_conv1)                          # output size 14×14×32

# conv2 layer
W_conv2 = weight_variable([5, 5, 32, 64])  # patch 5×5，in size 32，out size 64
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2)+b_conv2)   # output size 14×14×64
h_pool2 = max_pool_2x2(h_conv2)                          # output size 7×7×64

# func1 layer
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])         # [n_samples,7,7,64] ->> [n_samples,7*7*64]
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# func2 layer
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2)+b_fc2)

# 损失函数
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys*tf.log(prediction), reduction_indices=[1]))

# 训练优化器
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

sess = tf.Session()
sess.run(tf.initialize_all_variables())

for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)     # 从整体数据中取出100个（不采用整体学习的方式，提高学习速度）
    sess.run(train_step, feed_dict={xs:batch_xs, ys:batch_ys, keep_prob:0.5})
    if i%50 == 0:
        print(compute_accuracy(mnist.test.images, mnist.test.labels))

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
0.1482
0.804
0.8787
0.9103
0.9228
0.9298
0.9385
0.9451
0.9481
0.9492
0.9556
0.9566
0.961
0.9609
0.9612
0.964
0.9642
0.9641
0.9651
0.9689


### tf.truncated_normal(shape, mean, stddev)，生成正态分布随机数
**说明**：
截断的产生正态分布的随机数，即随机数与均值的差值大于两倍的标准差，则重新生成。
* shape表示生成张量的维度
* mean表示均值
* stddev表示标准差

In [4]:
import tensorflow as tf;
import numpy as np;
import matplotlib.pyplot as plt;
 
c = tf.truncated_normal(shape=[5,5], mean=0, stddev=1)
 
with tf.Session() as sess:
    print(sess.run(c))

[[-0.40872586 -1.4420594   0.79212666 -0.07308755 -0.09529229]
 [-1.488096    0.33658066 -0.66655415 -1.4347384   1.4839036 ]
 [ 1.611575    1.1346846  -1.505412    1.3316833  -0.69946456]
 [-1.063847   -0.48559287  0.73066175 -0.58961767 -0.39298633]
 [ 0.10171129 -0.3605982   0.36176232  0.12893438 -1.0475997 ]]


### tf.constant(value, shape)，生成常量

In [5]:
import tensorflow as tf;
import numpy as np;
import matplotlib.pyplot as plt;
 
c = tf.constant(-1, shape=[2,3])
 
with tf.Session() as sess:
    print(sess.run(c))

[[-1 -1 -1]
 [-1 -1 -1]]


#### TensorFlow报错：InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_2' with dtype float

**如下**：
> InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_2' with dtype float
>
> 	 [[node Placeholder_2 (defined at <ipython-input-19-554db1d93dc1>:40) ]]

**分析**：sess.run(train_step, feed_dict = {xs: batch_xs, ys: batch_ys}) 中 feed_dict 缺少参数

**解决方式**：补充 keep_prob:0.5，如下所示

> sess.run(train_step, feed_dict = {xs: batch_xs, ys: batch_ys, keep_prob: 0.5}) 